In [1]:
import h3
import geopandas as gpd

from urllib.request import urlopen
from shapely.geometry import MultiPolygon, Polygon

Exhibit uses [H3](https://eng.uber.com/h3/) hexes as a base unit in the generation of geospatial data from which individual points and their latitude / longitude pairs are sampled. This notebook walks through the steps of transforming geospatial data from the shapefile (or geojson) format to a list of H3 hexes. Once you have the H3 list, you can insert it into Exhibit's `anon.db` SQLite database and use the table name as reference in the specification.

For this scenario, we will generate H3 hexes from a shapefile of Scotland's DataZones and augment the lookup with a rough population estimate so that our generated data follows the population spread.

In [2]:
dz_shapefile = urlopen("https://maps.gov.scot/ATOM/shapefiles/SG_DataZoneBdry_2011.zip")

In [3]:
dz_df = gpd.read_file(dz_shapefile).set_crs('EPSG:27700', allow_override=True)

In [4]:
dz_df.head()

,DataZone,Name,TotPop2011,ResPop2011,HHCnt2011,StdAreaHa,StdAreaKm2,Shape_Leng,Shape_Area,geometry
0,S01006506,Culter - 01,872,852,424,438.880218,4.388801,11801.872345,4.388802e+06,"POLYGON ((383285.265 800510.607, 383348.492 80..."
1,S01006507,Culter - 02,836,836,364,22.349739,0.223498,2900.406362,2.217468e+05,"POLYGON ((383527.919 801536.276, 383541.089 80..."
2,S01006508,Culter - 03,643,643,340,27.019476,0.270194,3468.761949,2.701948e+05,"POLYGON ((383473.000 801227.000, 383597.000 80..."
3,S01006509,Culter - 04,580,580,274,9.625426,0.096254,1647.461389,9.625426e+04,"POLYGON ((383976.659 801182.579, 383984.102 80..."
4,S01006510,Culter - 05,644,577,256,18.007657,0.180076,3026.111412,1.800766e+05,"POLYGON ((384339.000 801211.000, 384316.510 80..."


In [5]:
# reproject the original shapefile geometry to WGS84 which is what H3 uses
dz_df = dz_df.to_crs("EPSG:4326")

H3 hexes come at different resolutions: from 0 to 15. You can see the breakdown of what each resolution means in terms of the average hexagon area and average hexagon edge length [here](https://h3geo.org/docs/core-library/restable/). For our purposes resolution 8 is sufficient.

In [6]:
def process_polygon(poly, res=8):
    '''
    For convenience, we're picking up datazone-specific columns like
    datazone code and its population from the outer scope of the function.
    '''
    
    temp_result = []
    
    h3s = h3.polyfill_geojson(poly.__geo_interface__, res)

    for h in h3s:

        temp_result.append((
            dz,
            pop,
            h,
            Polygon(h3.h3_to_geo_boundary(h, True))
        ))
    
    return temp_result

In [7]:
result = []

for _, dz, pop, geom in dz_df[["DataZone", "TotPop2011", "geometry"]].itertuples():
    
    if isinstance(geom, Polygon):
    
        result.extend(process_polygon(geom, res=8))
        
    # some datazones are MultiPolygons instead of simple Polygons!
    elif isinstance(geom, MultiPolygon):

        for poly in geom.geoms:
            
            result.extend(process_polygon(poly, res=8))

In [8]:
h3_df = gpd.GeoDataFrame(result, columns=["datazone", "dz_pop", "h3", "geometry"], crs="EPSG:4326")

In [9]:
h3_df.head()

,datazone,dz_pop,h3,geometry
0,S01006506,872,8819761443fffff,"POLYGON ((-2.26118 57.09362, -2.26871 57.09252..."
1,S01006506,872,8819761409fffff,"POLYGON ((-2.26990 57.08828, -2.27743 57.08717..."
2,S01006506,872,8819761441fffff,"POLYGON ((-2.26632 57.10100, -2.27385 57.09990..."
3,S01006506,872,881976140dfffff,"POLYGON ((-2.28377 57.09031, -2.29130 57.08920..."
4,S01006506,872,8819761429fffff,"POLYGON ((-2.30636 57.08699, -2.31390 57.08588..."


In [10]:
# create a population figure for each hex by dividing the DZ population by the count of constituent hexes
# this is a very rough approximation of how the area of a DZ is related to the its population: a city DZ can
# be a single hex with ~1000 population, but a rural DZ with ~1000 population might be made up of 4 hexes.
h3_df["h3_pop"] = h3_df.groupby("datazone")["dz_pop"].transform(lambda x: x / len(x))

To see the result of your transformation, you can export the dataframe to GeoJSON and use your favourite mapping tool like Tableau, QGIS or ArcPro to visualise the hexagons. You can also use a free web-based tool [kepler.gl](https://kepler.gl/)

`h3_df.to_file("h3.geojson", driver="GeoJSON", index=False)`

In order for Exhibit to properly recognize your lookup table, it **must** have a column called `h3` with H3 ids. If you would like to weight the samples (so that sampled points reflect highly populated areas, for example), please ensure you include the appropriate weights column together with the `h3` column and reference it in the specification. By default, the distribution is set to uniform, but by specifying the weights column name, you can easily change that.

To insert the h3 lookup table into `anon.db` you can use the `db_util.py` provided in the `exhibit/db/` folder. Once you've exported your `h3_df` to a `.csv` file called, for example, `geo_h3_datazones.csv`, all you need to do is type the following in the terminal: `python db_util.py --insert geo_h3_datazones.csv` assuming your `csv` file is in the same folder as `db_util.py`.